# M.Lytova, M.Spanner, I.Tamblyn. *Deep learning and high harmonic generation* (2020)
## Codes for Section IV.A.1 : *Symmetric diatomic molecule*

##Headers and constants

In [ ]:
from google.colab import files
import numpy as np
import tensorflow as tf
from keras.layers import Input, Dense, Dropout, Conv1D, MaxPooling1D, UpSampling1D, Flatten, Reshape
from keras.models import Model
from keras.optimizers import Nadam, Adam
from tensorflow.keras import initializers
from keras import objectives
from keras.losses import mean_squared_error
from keras.callbacks import TensorBoard
from keras import backend as K
import argparse
import matplotlib.pyplot as plt
import time

In [ ]:
PI = 3.14159265359

t_n_points = 4096   # number of nodes in time
t_n = np.linspace(0, 800, t_n_points)/41.341    # grid in time, Tmax = 800 a.u. = 19.35 fs

n_train = 30000   # training set size
n_test = 1000     # testing set size

##Loading a training set

In [ ]:
param_train = np.zeros((n_train, 3))
path2param = f"/hhg_reduced/param.dat"
param_train = np.loadtxt(path2param, delimiter = ",", max_rows = n_train) 

In [ ]:
y_train = np.zeros((n_train, t_n_points))
path2load0 = f"/hhg_reduced/hhg"

tic = time.perf_counter()

for i in range(n_train): 
    path2load = path2load0 + str(i+1) + '.dat'    
    load_data = np.loadtxt(path2load)
    y_train[i] = load_data[0:t_n_points] * np.sin(PI*t_n/Tmax) 
    if (round(i/1000)==i/1000):
        print(i)        

toc = time.perf_counter()
print(f"Training set preparation time {toc - tic:0.4f} seconds")   

In [ ]:
def plot_train_example(i):
    plt.figure(figsize=(16,5), constrained_layout=False)    
    plt.plot(t_n, y_train[i], color='green')
    plt.xlim(0, 20)
    plt.xticks(np.arange(0, 20, 2.0))
    plt.grid()
    plt.show() 
    plt.close()

### Drawing of a randomly chosen $d_k(t)$

In [ ]:
def plot_train_example(i):
    plt.figure(figsize=(12,4), constrained_layout=False)    
    plt.plot(t_n, y_train[i], color='green')
    plt.title('theta = ' + str(round(param_train[i,0]*180/PI,2)) + ",  R = " + str(round(param_train[i,1],2)) + \
              "a.u. , I = " + str(round((param_train[i,2]/5.338027e-2)**2,2)) + "e14 W/cm^2", fontsize=16)
    plt.xlabel('$t$', fontsize=14)
    plt.ylabel('$y(t)$', fontsize=14)
    plt.xticks(np.arange(0, Tmax, 2.0))
    plt.grid()    
    plt.show() 
    plt.close()

In [ ]:
i_show = np.random.randint(0, n_train-1)

plot_train_example(i_show)

##Loading a testing set

In [ ]:
param_test = np.zeros((n_test, 3))
path2param = f"/hhg_reduced/param.dat"
param_test = np.loadtxt(path2param, delimiter = ",", skiprows = n_train, max_rows = n_test) 

In [ ]:
y_test = np.zeros((n_test, t_n_points))
path2load0 = f"/hhg_reduced/hhg"

for i in range(n_test): 
    path2load = path2load0 + str(i+1+n_train) + '.dat'    
    load_data = np.loadtxt(path2load)
    y_test[i] = load_data[0:t_n_points] * np.sin(PI*t_n/Tmax) 
   

##Normalizing before training

In [ ]:
max_E0 = np.amax(param_train[:,2])
min_E0 = np.amin(param_train[:,2])
param_train_norm = (param_train-[0, 1.5, min_E0])/[PI/2, 2.5, (max_E0-min_E0)]
param_test_norm = (param_test-[0, 1.5, min_E0])/[PI/2, 2.5, (max_E0-min_E0)]
y_max = 0.3 
y_train_norm = y_train/y_max
y_test_norm = y_test/y_max

## Model

In [ ]:
inputs = Input(shape=(3,))

x = Dense(16, activation='tanh')(inputs) 
x = Dense(64, activation='tanh')(x) 
x = Reshape((64, 1))(x)
x = Conv1D(8, 4, activation='tanh', padding='same')(x)
x = Conv1D(8, 4, activation='tanh', padding='same')(x)
x = UpSampling1D(2)(x)
x = Conv1D(8, 4, activation='tanh', padding='same')(x)
x = Conv1D(8, 4, activation='tanh', padding='same')(x)
x = UpSampling1D(2)(x)
x = Conv1D(8, 4, activation='tanh', padding='same')(x)  
x = Conv1D(8, 4, activation='tanh', padding='same')(x) 
x = UpSampling1D(2)(x)  
x = Conv1D(8, 4, activation='tanh', padding='same')(x)                                                                      
x = Conv1D(1, 4, activation='tanh', padding='same')(x)
x = Flatten()(x)
x = Dense(512, activation='tanh')(x) 
outputs = Dense(t_n_points, activation='tanh')(x)

ModelGen = Model(inputs, outputs)
opt = Adam(lr=0.0005, amsgrad=True)
ModelGen.compile(optimizer=opt, loss='mean_squared_error') 

print(ModelGen.summary())

##Training

*   Training set: 30,000
*   Testing set: 1,000

In [ ]:
def plot_losses2():
    plt.figure(figsize=(8,4))
    plt.plot(np.log10(loss_sum),color='blue')
    plt.plot(np.log10(val_loss_sum),color='red')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['training', 'validation'], loc='upper right')
    plt.show()

In [ ]:
tic = time.perf_counter()

for n in range(3, 10):        # training in a cycle with increasing batch size  

      batch_size = 2**n       

      history = ModelGen.fit(param_train_norm, y_train_norm, 
                             epochs=300,
                             batch_size=batch_size,
                             shuffle=True,
                             validation_data=(param_test_norm, y_test_norm))
      
      path = f"/model_thetaRI/model_1" 
      ModelGen.save(path) 

      loss_save = history.history['loss']
      val_loss_save = history.history['val_loss']
      if n > 3:
            loss_sum = np.concatenate((loss_sum, loss_save), axis = 0)
            val_loss_sum = np.concatenate((val_loss_sum, val_loss_save), axis = 0)
      else:
            loss_sum = loss_save
            val_loss_sum = val_loss_save           
      
      plot_losses2()

toc = time.perf_counter()
print(f"Execution time {toc - tic:0.4f} seconds")
    

##Training and validation losses

In [ ]:
plot_losses2()  

##Prediction

In [ ]:
prediction = y_max*ModelGen.predict(param_test_norm)

##Function to draw the test and predicted examples

In [ ]:
def plot_examples(i1, i2):    
    fig = plt.subplots(2,1,figsize=(12,8),constrained_layout=False)
    plt.suptitle('Examples: Test points and prediction', fontsize=16)
    plt.subplot(211)
    plt.title('theta = ' + str(round(param_test[i1,0]*180/PI,2)) + ",  R = " + str(round(param_test[i1,1],2)) + \
              "a.u., I = " + str(round((param_test[i1,2]/5.338027e-2)**2,2)) + "e14 W/cm^2", fontsize=16)
    plt.scatter(t_n, y_test[i1], color="blue", s = 0.5)
    plt.plot(t_n, prediction[i1], color="red", linewidth = 1)
    plt.ylabel('$d(t)$, a.u.', fontsize=16)
    plt.xlim(0, 20)
    plt.xticks(np.arange(0, Tmax, 2.0))
    plt.grid()
    plt.subplot(212)
    plt.title('theta = ' + str(round(param_test[i2,0]*180/PI,2)) + ",  R = " + str(round(param_test[i2,1],2))  + \
              "a.u., I = " + str(round((param_test[i2,2]/5.338027e-2)**2,2)) + "e14 W/cm^2", fontsize=16)
    plt.scatter(t_n, y_test[i2], color="blue", s = 0.5)
    plt.plot(t_n, prediction[i2], color="red", linewidth = 1)  
    plt.ylabel('$d(t)$, a.u.', fontsize=16)
    plt.xlabel('$t$, fs', fontsize=16)
    plt.xlim(0, 20)
    plt.xticks(np.arange(0, Tmax, 2.0))
    plt.grid()  
    plt.show() 
    plt.close()    

##Comparison of arbitrary $y_{test}$ (blue) and $y_{decoded}$ (red)

In [ ]:
i_show1 = np.random.randint(0, n_test-1)
i_show2 = np.random.randint(0, n_test-1)

plot_examples(i_show1, i_show2)